In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model


# importing training data
training_data = pd.read_csv("/content/training_data_EduBot_v2.txt")

# preprocessing training data
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

# preprocessing target variable(tags)
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

# creating DNN
edubot = Sequential()
edubot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
edubot.add(Dense(8))
edubot.add(Dense(8))
edubot.add(Dense(6))
edubot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))
edubot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# fitting DNN
edubot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=50, batch_size=32)

# saving model file
save_model(edubot, "EduBot_v2")



Epoch 1/50
28/28 [==============================] - 1s 2ms/step - loss: 3.6134 - accuracy: 0.0673
Epoch 2/50
28/28 [==============================] - 0s 2ms/step - loss: 3.5440 - accuracy: 0.1300
Epoch 3/50
28/28 [==============================] - 0s 2ms/step - loss: 3.4391 - accuracy: 0.1513
Epoch 4/50
28/28 [==============================] - 0s 2ms/step - loss: 3.2908 - accuracy: 0.1648
Epoch 5/50
28/28 [==============================] - 0s 2ms/step - loss: 3.1057 - accuracy: 0.2410
Epoch 6/50
28/28 [==============================] - 0s 2ms/step - loss: 2.8876 - accuracy: 0.2892
Epoch 7/50
28/28 [==============================] - 0s 2ms/step - loss: 2.6582 - accuracy: 0.2926
Epoch 8/50
28/28 [==============================] - 0s 2ms/step - loss: 2.4440 - accuracy: 0.3004
Epoch 9/50
28/28 [==============================] - 0s 2ms/step - loss: 2.2574 - accuracy: 0.3139
Epoch 10/50
28/28 [==============================] - 0s 2ms/step - loss: 2.0944 - accuracy: 0.3475
Epoch 11/50
28/28 [

In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import load_model
import json
import random
#Let’s load data, model, and responses

# importing training data
training_data = pd.read_csv("/content/training_data_EduBot_v2.txt")
# loading model
chatbot = load_model("EduBot_v2")


In [ ]:
# loading responses
responses = json.load(open("/content/responses (1).json"))

In [ ]:
# fitting TfIdfVectorizer with training data to preprocess inputs
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
vectorizer.fit(training_data["patterns"])

TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

In [ ]:
# fitting LabelEncoder with target variable(tags) for inverse transformation of predictions
le = LabelEncoder()
le.fit(training_data["tags"])

LabelEncoder()

In [ ]:
# transforming input and predicting intent
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str.lower()]).toarray()
    predicted_proba = edubot.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    return predicted_tag

In [ ]:
# defining chat function
def start_chat():
    print("---------------  AI Chat bot  ---------------")
    print("Ask any queries...")
    print("I will try to understand you and reply...")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        if inp == "EXIT":
            break
        else:
            if inp:
                tag = predict_tag(inp)
                response = random.choice(responses[tag])
                print("Response... : ", response)
            else:
                pass
# calling chat function to start chatting
start_chat()

---------------  AI Chat bot  ---------------
Ask any queries...
I will try to understand you and reply...
Type EXIT to quit...
1/1 [==============================] - 0s 97ms/step
Response... :  Hi! I'm ready to answer all your questions.
1/1 [==============================] - 0s 20ms/step
Response... :  Admission to the B.Tech / 5-year Integrated M.Tech programs will be conducted under two streams.
(1) Stream 1 (based on JEE(Main) 2021 scores and +2 Marks) for 70% of seats.
(2) Stream 2 (based on +2 marks) for 30% of seats.
Please visit https://www.sastra.edu/admissions.html for more info.
